In [1]:
from __future__ import print_function

from osgeo import gdal
import rasterio
import pandas as pd
import os 
import boto3

In [2]:
try:
    os.makedirs("images/niihau", 0755)  
except OSError:
    print("folder already exists")

try:
    os.makedirs("niihau", 0755)  
except OSError:
    print("folder already exists")


In [3]:

root_folder = os.getcwd()
root_folder = "/tmp"
print(root_folder)

/tmp


In [9]:
not_default_keys = True

if not_default_keys:
    s3_client = boto3.client('s3',
                aws_access_key_id=os.environ.get('hawaii_debris_access_key'),
                aws_secret_access_key=os.environ.get('hawaii_debris_secret_key'))
    s3_resource = boto3.resource('s3',aws_access_key_id=os.environ.get('hawaii_debris_access_key'),
                aws_secret_access_key=os.environ.get('hawaii_debris_secret_key'))
else:
    s3_client = boto3.client('s3')
    s3_resource = boto3.resource('s3')
    
    
bucket_name = 'hawaii-marine-debris'
file_path_template = "{}/jpg/{}"
downloaded_tile = set()
csv_rows = []
islands= ['niihau']#'lanai','bigisland',''
local_tmp_folder = "/tmp/{}"

annotatino_s3key_path = "{island}/annotations.csv"
geojson_keys = [
    "niihau/final_marine_debris_database_NI_UTM4N_boxes.geojson",
    "niihau/niihau_tileindex.geojson",    
]


s3_resource.Bucket(bucket_name).download_file("niihau/final_marine_debris_database_NI_UTM4N_boxes.geojson", "{}/final_marine_debris_database_NI_UTM4N_boxes.geojson".format(root_folder))
s3_resource.Bucket(bucket_name).download_file("niihau/niihau_tileindex.geojson", "{}/niihau_tileindex.geojson".format(root_folder))
s3_resource.Bucket(bucket_name).download_file("niihau/annotations.csv", "{}/niihau/annotations.csv".format(root_folder))


In [10]:
s3_resource.Bucket(bucket_name).download_file("niihau/annotations.csv", "{}/niihau/annotations.csv".format(root_folder))
existing_annotation_df = pd.read_csv("{}/niihau/annotations.csv".format(root_folder))
existing_annotation_df['file_path'] = existing_annotation_df['s3_key'].replace({'niihau/jpg': '{}/niihau/images'.format(root_folder)}, inplace=False, regex=True)
existing_annotation_df

,s3_key,x0,y0,x1,y1,label,file_path
0,niihau/jpg/1_1920_06_12.jpg,940,393,990,343,P,/srv/notebooks/niihau/images/1_1920_06_12.jpg
1,niihau/jpg/1_1920_06_13.jpg,140,393,190,343,P,/srv/notebooks/niihau/images/1_1920_06_13.jpg
2,niihau/jpg/1_1920_07_13.jpg,242,529,292,479,P,/srv/notebooks/niihau/images/1_1920_07_13.jpg
3,niihau/jpg/1_1920_07_12.jpg,638,801,738,701,P,/srv/notebooks/niihau/images/1_1920_07_12.jpg
4,niihau/jpg/1_1920_08_13.jpg,187,133,237,83,P,/srv/notebooks/niihau/images/1_1920_08_13.jpg
5,niihau/jpg/1_1920_05_11.jpg,528,991,578,941,P,/srv/notebooks/niihau/images/1_1920_05_11.jpg
6,niihau/jpg/1_1920_06_11.jpg,528,191,578,141,P,/srv/notebooks/niihau/images/1_1920_06_11.jpg
7,niihau/jpg/1_1920_08_12.jpg,913,376,963,326,P,/srv/notebooks/niihau/images/1_1920_08_12.jpg
8,niihau/jpg/1_1920_08_14.jpg,184,659,234,609,P,/srv/notebooks/niihau/images/1_1920_08_14.jpg
9,niihau/jpg/1_1920_07_14.jpg,326,995,376,945,P,/srv/notebooks/niihau/images/1_1920_07_14.jpg


In [17]:
##add negative examples

In [18]:
def download_from_s3(s3_key):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_folder.format(basename)
    s3_resource.Bucket(bucket_name).download_file(s3_key, local_file_path)
    return local_file_path

def convert_tif_to_jpeg(tiff_image_path):
    src = rasterio.open(tiff_image_path)
    jpg_path = tiff_image_path.replace(".tif", ".jpg")
    data = src.read()
    profile = src.profile
    result = data.astype(rasterio.uint8, casting='unsafe', copy=False)
    profile.update(driver='jpeg')
    with rasterio.open(jpg_path, 'w', **profile) as dst:
        dst.write(result)
    return jpg_path
                
def upload_to_s3(local_file_path, s3_path):
    s3_client.upload_file(local_file_path, bucket_name, s3_path)

def find_min_xy(coordinates):
    x0 = min([i[0] for i in coordinates])
    x1 = max([i[0] for i in coordinates])
    y0 = min([i[1] for i in coordinates])
    y1 = max([i[1] for i in coordinates])
    return x0,y0,x1,y1


def get_pixel_coordinates(image_path, coordinates):
    driver = gdal.GetDriverByName('GTiff')
    dataset = gdal.Open(image_path)
    band = dataset.GetRasterBand(1)

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize

    transform = dataset.GetGeoTransform()

    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    data = band.ReadAsArray(0, 0, cols, rows)

    x0, y0, x1, y1 = coordinates
    x0p = int((x0 - xOrigin) / pixelWidth)
    y0p = int((yOrigin - y0 ) / pixelHeight)
    x1p = int((x1 - xOrigin) / pixelWidth)
    y1p = int((yOrigin - y1 ) / pixelHeight)
    return x0p, y0p, x1p, y1p

### Parse geojson 

In [19]:
#download all necessary geojson files:


In [11]:
import json
from pprint import pprint
#list of all debris annotated from UoH
with open("{}/final_marine_debris_database_NI_UTM4N_boxes.geojson".format(root_folder)) as f:
    geo_json = json.load(f)
len(geo_json['features'])
debris_boxes = geo_json['features']


with open("{}/niihau_tileindex.geojson".format(root_folder)) as f:
    tiles_geo_json = json.load(f)
len(tiles_geo_json['features'])
tiles_features = tiles_geo_json['features']



In [12]:
debris_boxes[0]

{u'geometry': {u'coordinates': [[[390411.34116614977, 2432815.3680491154],
    [390412.34116614977, 2432815.3680491154],
    [390412.34116614977, 2432816.3680491154],
    [390411.34116614977, 2432816.3680491154],
    [390411.34116614977, 2432815.3680491154]]],
  u'type': u'Polygon'},
 u'properties': {u'comment': None,
  u'island': u'NI',
  u'lat': 21.996464,
  u'long': -160.061616,
  u'max_size_meters': 0.5,
  u'max_size_s': u'0.5',
  u'min_size_meters': 0.01,
  u'observer': u'AO',
  u'pt_id': 1,
  u'segment': 1,
  u'size': 1,
  u'type': u'P',
  u'unique_pt_id': u'NI-001-0001'},
 u'type': u'Feature'}

In [13]:
#filter out specific data that we are looking for
target_islands = ["NI"]
objects = ['T','V','N','B','M','P','C']
geo_json_in_search = []
set_of_objs = set()
for d in debris_boxes:
    if d['properties']['type'] in objects and d['properties']['island'] in target_islands:
        geo_json_in_search.append(d)
    set_of_objs.add(d['properties']['type'])
    
print("# of debris :{}".format(len(geo_json_in_search)))
print(set_of_objs)
# Debris categories:
# B = Buoys and floats
# C = Cloth	
# F = Foam 
# L = Line (single pieces of rope, not net)
# M = Metal
# N = Net
# P = Plastic
# T = Tire
# W = Processed wood
# V = Vessel
# O = Other 

# of debris :6797
set([u'C', u'B', u'p', u'F', u'M', u'L', u'O', u'N', u'P', u'T', u'W', u'V'])


In [14]:

#find corresponding tile
tile_with_debris = []
print("searching {}".format(len(geo_json_in_search)))


searching 6797


In [ ]:

bucket_name = 'hawaii-marine-debris'

s3key_path_template = "niihau/jpg/{}"
downloaded_tile = set()
csv_rows = []

print("Search for tile image based on annotation geojson")
for debris in geo_json_in_search:
    found_debris = False
    target = find_min_xy(debris['geometry']['coordinates'][0])
    for tile in tiles_features:
        x0,y0,x1,y1 = find_min_xy(tile['geometry']['coordinates'][0]) 
        if x0 < target[0] and y0 < target[1] and  x1 > target[2] and y1 > target[3]:
            json = {'label': d['properties']['type'],
             'annotation': target,
             'image_s3':tile['properties']['s3_path'],
            }
            tile_with_debris.append(json)
            print("found--{}".format(len(tile_with_debris)),end='\r')
            found_debris = True
            break
    if found_debris:
        continue
    
print("found--{}".format(len(tile_with_debris)),end='\r')
#just find the first one for now. and let's think about how to deal with second images later.

Search for tile image based on annotation geojson


In [ ]:
tile_with_debris[0]

In [ ]:
try:
    s3_client.head_object(
        Bucket=bucket_name,
        Key='niihau/1_1920/1_1920_06_12.tif_not_exit'
    )
except:
    print("no such file yet")

In [ ]:

print("convert image to JPEG, and convert bounding box to pixels")

for debris in tile_with_debris:
    s3filepath = debris['image_s3'].replace('https://s3-us-west-2.amazonaws.com/hawaii-marine-debris/','')
    tmpfile = "{}/images/niihau/{}".format(root_folder, os.path.basename(s3filepath))

    if tmpfile not in downloaded_tile:
        s3_resource.Bucket(bucket_name).download_file(s3filepath, tmpfile)
        downloaded_tile.add(tmpfile)
    
    pixel_coordinates = get_pixel_coordinates(tmpfile, debris['annotation'])
    jpg_path = convert_tif_to_jpeg(tmpfile)    
    s3_key = s3key_path_template.format(os.path.basename(jpg_path))
    print("Uploaded %s: %s -> %s -> %s" % (len(csv_rows),tmpfile, jpg_path, s3_key), end='\r')
    if tmpfile not in downloaded_tile:
        try:
            s3_client.head_object(
                Bucket=bucket_name,
                Key=s3_key
            )
        except:
            upload_to_s3(jpg_path, s3_key)
    csv_rows.append((s3_key,)+ pixel_coordinates+(debris['label'],))
    #free up storage once it's uploaded
#     os.remove(jpg_path)
#     os.remove(tmpfile)

In [ ]:
len(csv_rows)

In [104]:
import csv

with open('annotated_marine_debris.csv','wb') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['s3_key','x0', 'y0','x1','y1', 'label'])
    for row in csv_rows:
        csv_out.writerow(row)

In [105]:
s3_client.upload_file('annotated_marine_debris.csv', bucket_name, "niihau/annotations.csv")